To demonstrate charting using the seaborn library

In [1]:
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import seaborn as sns



In [8]:
#Read in NZ wholesale electricity prices from 2012 to 2021
prices = pd.read_csv('Wholesale_price_trends_20220820114423.csv', \
  header=8)
prices.head()

,Period start,Period end,Region ID,Region,Price ($/MWh)
0,01/01/2012,02/01/2012,OTA2201,OTA2201 - Otahuhu,48.13
1,01/01/2012,02/01/2012,WKM2201,WKM2201 - Whakamaru,44.29
2,01/01/2012,02/01/2012,RDF2201,RDF2201 - Redclyffe,43.76
3,01/01/2012,02/01/2012,SFD2201,SFD2201 - Stratford,45.24
4,01/01/2012,02/01/2012,HAY2201,HAY2201 - Haywards,45.06


In [ ]:
prices.info()

In [6]:
#Convert columns to useful pandas datatypes
prices['Period start'] = pd.to_datetime(prices['Period start'],dayfirst=True )
prices['Period end'] = pd.to_datetime(prices['Period end'],dayfirst=True )

prices = prices.astype({ \
  'Region ID': 'category', \
  'Region': 'category'})

prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14619 entries, 0 to 14618
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Period start   14619 non-null  datetime64[ns]
 1   Period end     14619 non-null  datetime64[ns]
 2   Region ID      14619 non-null  category      
 3   Region         14619 non-null  category      
 4   Price ($/MWh)  14619 non-null  float64       
 5   week           14619 non-null  UInt32        
 6   month          14619 non-null  int64         
 7   year           14619 non-null  int64         
 8   day            14619 non-null  int64         
dtypes: UInt32(1), category(2), datetime64[ns](2), float64(1), int64(3)
memory usage: 786.1 KB


In [7]:
prices['week'] = prices['Period start'].dt.isocalendar().week
prices['month'] = prices['Period start'].dt.month
prices['year'] = prices['Period start'].dt.year


In [5]:
prices[prices['Region ID']=='OTA2201'].head(20)

,Period start,Period end,Region ID,Region,Price ($/MWh),week,month,year,day
0,2012-01-01,02/01/2012,OTA2201,OTA2201 - Otahuhu,48.13,52,1,2012,1
8,2012-01-02,03/01/2012,OTA2201,OTA2201 - Otahuhu,52.67,1,1,2012,2
16,2012-01-03,04/01/2012,OTA2201,OTA2201 - Otahuhu,50.41,1,1,2012,3
24,2012-01-04,05/01/2012,OTA2201,OTA2201 - Otahuhu,36.30,1,1,2012,4
32,2012-01-05,06/01/2012,OTA2201,OTA2201 - Otahuhu,29.00,1,1,2012,5
40,2012-01-06,07/01/2012,OTA2201,OTA2201 - Otahuhu,51.96,1,1,2012,6
48,2012-01-07,08/01/2012,OTA2201,OTA2201 - Otahuhu,45.03,1,1,2012,7
56,2012-01-08,09/01/2012,OTA2201,OTA2201 - Otahuhu,28.52,1,1,2012,8
64,2012-01-09,10/01/2012,OTA2201,OTA2201 - Otahuhu,67.56,2,1,2012,9
72,2012-01-10,11/01/2012,OTA2201,OTA2201 - Otahuhu,86.12,2,1,2012,10
